In [1]:
# Accuracy assessment script for IoU calculation between reference and predicted shapefiles
import os
import geopandas as gpd
import pandas as pd

# === Root folder ===
root_dir = r"C:\Users\spn733\Work\CSA_Segmentation\CSA_Field_Boundary_Segmentation\6. Data\Accuracy assessment\SK"
summary_path = os.path.join(root_dir, "iou_summary.csv")

iou_records = []  # (subfolder, iou, intersection_area, union_area, crs)

# === Function to compute UTM zone from centroid ===
def get_utm_crs(gdf):
    gdf_ll = gdf.to_crs("EPSG:4326")
    gdf_ll = gdf_ll[gdf_ll.geometry.is_valid & ~gdf_ll.geometry.is_empty]
    if gdf_ll.empty:
        raise ValueError("No valid geometry for CRS calculation.")
    
    centroid = gdf_ll.geometry.union_all().centroid
    lon, lat = centroid.x, centroid.y
    if not (-180 <= lon <= 180 and -90 <= lat <= 90):
        raise ValueError(f"Invalid centroid coordinates: lon={lon}, lat={lat}")
    
    utm_zone = int((lon + 180) / 6) + 1
    epsg = 32600 + utm_zone if lat >= 0 else 32700 + utm_zone
    return f"EPSG:{epsg}"

# === Loop through subfolders ===
for subfolder in os.listdir(root_dir):
    subfolder_path = os.path.join(root_dir, subfolder)
    if not os.path.isdir(subfolder_path):
        continue

    shp_files = [f for f in os.listdir(subfolder_path) if f.endswith('.shp')]
    ref_file = next((f for f in shp_files if f.startswith('boundary')), None)
    pred_file = next((f for f in shp_files if f != ref_file), None)

    if ref_file is None or pred_file is None:
        print(f"⚠️ Skipping {subfolder}: missing shapefiles.")
        continue

    ref_path = os.path.join(subfolder_path, ref_file)
    pred_path = os.path.join(subfolder_path, pred_file)

    try:
        ref_gdf = gpd.read_file(ref_path)
        pred_gdf = gpd.read_file(pred_path)

        utm_crs = get_utm_crs(ref_gdf)
        ref_gdf = ref_gdf.to_crs(utm_crs)
        pred_gdf = pred_gdf.to_crs(utm_crs)

        ref_union = ref_gdf.geometry.union_all()
        pred_union = pred_gdf.geometry.union_all()

        intersection = ref_union.intersection(pred_union)
        union = ref_union.union(pred_union)

        intersection_area = intersection.area
        union_area = union.area
        iou = intersection_area / union_area if union_area > 0 else 0

        # Write individual result
        txt_path = os.path.join(subfolder_path, "iou_result.txt")
        with open(txt_path, "w") as f:
            f.write(f"IoU: {iou:.4f}\n")
            f.write(f"Intersection area: {intersection_area:.2f} m²\n")
            f.write(f"Union area: {union_area:.2f} m²\n")
            f.write(f"CRS used: {utm_crs}\n")

        # Save to summary list
        iou_records.append({
            "Subfolder": subfolder,
            "IoU": round(iou, 4),
            "Intersection_area_m2": round(intersection_area, 2),
            "Union_area_m2": round(union_area, 2),
            "CRS": utm_crs
        })

        print(f"✅ {subfolder}: IoU = {iou:.4f}")

    except Exception as e:
        print(f"❌ Error in {subfolder}: {e}")

# === Save all IoUs to CSV ===
if iou_records:
    df = pd.DataFrame(iou_records)
    df.to_csv(summary_path, index=False)
    print(f"\n📄 Saved combined summary to: {summary_path}")

    # Print final stats
    mean_iou = df["IoU"].mean()
    print("\n📊 Overall Summary:")
    print(df.to_string(index=False))
    print(f"\n✅ Overall Mean IoU = {mean_iou:.4f}")
else:
    print("⚠️ No valid results found. No summary written.")


✅ Box_0: IoU = 0.8319
✅ Box_1: IoU = 0.8887
✅ Box_2: IoU = 0.6071
✅ Box_3: IoU = 0.8779
✅ Box_4: IoU = 0.8932
✅ Box_5: IoU = 0.8761
✅ Box_6: IoU = 0.8906

📄 Saved combined summary to: C:\Users\spn733\Work\CSA_Segmentation\CSA_Field_Boundary_Segmentation\6. Data\Accuracy assessment\SK\iou_summary.csv

📊 Overall Summary:
Subfolder    IoU  Intersection_area_m2  Union_area_m2        CRS
    Box_0 0.8319           12225026.54    14694917.68 EPSG:32613
    Box_1 0.8887           10286702.88    11574796.82 EPSG:32613
    Box_2 0.6071            5300167.43     8729765.42 EPSG:32613
    Box_3 0.8779           21849115.49    24888760.64 EPSG:32613
    Box_4 0.8932           14548465.80    16288640.97 EPSG:32613
    Box_5 0.8761           45830489.20    52312732.68 EPSG:32613
    Box_6 0.8906           10245970.55    11504810.39 EPSG:32613

✅ Overall Mean IoU = 0.8379
